<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyGROWAT2D</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>GRO</b>und<b style=color:red;>WAT</b>er</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyGROWAT2D.png"></td>
</tr>
</table>

----
# `pyGROWAT2D`

GROundWATer2D, a program package for flow in porous rocks.

----
# Numerical implementation

We implement the groundwater-flow equation based on a general form of an **advection-diffusion-reaction** equation,
which in 3D can be formulated as:
$$
\frac{\partial c}{\partial t}
= - \vec{u} \cdot \nabla c
- \nabla \cdot \vec{F}
+ M +R
$$
with:
- $c$ [-] a quantity (e.g. temperature, concentration, pressure, ...)
- $t$ [s] time
- $\vec{u}$ [m/s] velocity
- $\vec{F}$ [m/s] flux
- $M$ [1/s] source term
- $R$ [1/s] reaction term
- $\nabla$ [1/m] nabla operator

From this general **advection-diffusion-reaction** equation we can define equations for
- water pressure (head)
- water velocities
- temperature
- concentration
- dissolution

We will proceed with the equation for **groundwater flow**, described by the distribution of
the **hydraulic head** $h$.

----
## Groundwater-flow equation

The program `GROWAT1D` (GROundWATer2D) solves the **advection-diffusion-reaction** equation
without **diffusion** ($\vec{u} \cdot \nabla c=0$) and without **reaction** ($M=R=0$) in a two-dimensional
modeling domain for a porous aquifer under steady-state or transient flow conditions:
$$
\begin{array}{rcl}
 S \frac{\partial h}{\partial t} & = & - \nabla \cdot \vec{F} \\
 \vec{F} &=& - K \nabla h
\end{array}
$$
- The first equation is a conservation equation for flow, with $h$ [m] the hydraulic head,
$S$ [1/m] the specific storage,
$\vec{F}$ the Dary-flow velocity.
- The second equation is the **Darcy law** for laminar flow, 
with $K$ [m/s] the hydraulic conductivity.

These equations will be converted into a numerical algorithm next.

----
## Finite differences in 2D
We insert the fluxrate equation into the diffusion equation for a 2D geometry:
$$
S \frac{\partial h}{\partial t} 
= \frac{\partial}{\partial x} K \frac{\partial h}{\partial x}
+ \frac{\partial}{\partial y} K \frac{\partial h}{\partial y}
$$

----
## Explicit: FTCS scheme (forward time centered space)

We solve the above equation for hydraulic heads $h$, using a forward-time, centered-space method, `FTCS`.
Here, the derivatives in the transient diffusion equations are replaced by **forward differences in time** 
and **central differences is space**:
$$
\begin{array}{rcl}
S \frac{h_{i,j}^{n+1}-h_{i,j}^{n}}{\Delta t}
&=&
  \frac{K^n_{i+0.5,j} \frac{\partial h^n_{i+0.5,j}}{\partial x} - K^n_{i-0.5,j} \frac{\partial h^n_{i-0.5,j}}{\partial x}}{\Delta x} 
+ \frac{K^n_{i,j+0.5} \frac{\partial h^n_{i,j+0.5}}{\partial y} - K^n_{i,j-0.5} \frac{\partial h^n_{i,j-0.5}}{\partial y}}{\Delta y}\\
&=& 
  \frac{K^n_{i+0.5,j} h^n_{i+1,j} - \big( K^n_{i+0.5,j}+K^n_{i-0.5,j} \big) h^n_{i,j} + K^n_{i-0.5,j} h^n_{i-1,j} }{\Delta x^2}
+ \frac{K^n_{i,j+0.5} h^n_{i,j+1} - \big( K^n_{i,j+0.5}+K^n_{i,j-0.5} \big) h^n_{i,j} + K^n_{i,j-0.5} h^n_{i,j-1} }{\Delta y^2}
\end{array}
$$
For the hydraulic conductivities at half-step positions, we use averages:
$$
\begin{array}{rcl}
K_{left}   &=& \frac{1}{2} \big( K_{i-1,j} + K_{i,j} \big) \\
K_{right}  &=& \frac{1}{2} \big( K_{i,j} + K_{i+1,j} \big) \\
K_{bottom} &=& \frac{1}{2} \big( K_{i,j-1} + K_{i,j} \big) \\
K_{top}    &=& \frac{1}{2} \big( K_{i,j} + K_{i,j+1} \big) \\
\end{array}
$$

Solved for the next time step
$$\fbox{$
h_{{i,j}}^{n+1}
=
h_{{i,j}}^{n}
+ \frac{\Delta t}{S \Delta x^2} \big( K_{right} h^n_{i+1,j} - \big( K_{left}+K_{right} \big) h^n_{i,j} + K_{left} h^n_{i-1,j} \big)
+ \frac{\Delta t}{S \Delta y^2} \big( K_{top} h^n_{i,j+1} - \big( K_{bottom}+K_{top} \big) h^n_{i,j} + K_{bottom} h^n_{i,j-1}\big)
$}$$
The above equations is a set of $n$ equations for all nodes $(i,j)$ of the model domain.

In this **explicit scheme**, we calculate the hydraulic heads $h_{i,j}^{n+1}$ for the next time step $t^{n+1}$ from 
hydraulic heads $h_{i,j}^{n}$ of the previous time step $t^{n}$. We therefore can solve the above set of equations
**one-by-one**, and a simple **function** will be suitable.

This equation is implemented in the notebooks with the examples of the differential operators.

----
## Implicit:

Next, we extent the `FTCS` method to an **implicit** method. The method is called **implicit**,
because we will use information for the new time step $t^{n+1}$ by also using information of
the old and the new time steps:

----